# Analyse de la Longueur Totale des Lignes de Transport en Île-de-France


L'analyse présentée dans ce notebook vise à explorer et comprendre la longueur totale des lignes de transport par mode de transport et par exploitant en Île-de-France. Les données utilisées proviennent du dataset "Traces du réseau ferré IDF" disponible sur le site data.iledefrance-mobilites.fr à l'adresse https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/traces-du-reseau-ferre-idf/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B. Ce dataset contient des informations géospatiales et attributaires sur les traces du réseau ferré en Île-de-France, offrant ainsi une base solide pour diverses analyses liées aux infrastructures de transport.


Les objectifs de cette analyse sont multiples. Premièrement, il s'agit d'analyser la longueur totale des lignes de transport en fonction du mode de transport (RER, METRO, TRAMWAY, etc.) et de l'exploitant. Cela permettra de mieux comprendre la répartition et l'étendue des différents réseaux de transport en Île-de-France. Deuxièmement, cette analyse vise à fournir des insights sur la structure et la complexité du réseau ferré, ce qui peut être utile pour les planificateurs de transport, les décideurs politiques et les chercheurs.


## Méthodologie


La méthodologie employée dans cette analyse repose sur l'utilisation de DuckDB pour le traitement et l'analyse des données. Après avoir importé les données dans DuckDB, une requête SQL a été exécutée pour calculer la longueur totale des lignes de transport par mode de transport et par exploitant. Cette requête a utilisé les fonctions spatiales de DuckDB, telles que `ST_Length` et `ST_GeomFromGeoJSON`, pour extraire et traiter les géométries des lignes de transport à partir des données JSON. Les résultats ont ensuite été triés et visualisés à l'aide de Plotly Express pour créer un graphique à barres interactif illustrant la longueur totale des infrastructures par mode de transport et exploitant.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/traces-du-reseau-ferre-idf/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "mode", 
  "exploitant", 
  SUM(ST_Length(ST_GeomFromGeoJSON("Geo Shape"))) AS total_length,
  COUNT(*) AS num_lines
FROM 
  loaded_dataset
GROUP BY ALL
ORDER BY total_length DESC """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 15 lignes


,mode,exploitant,total_length,num_lines
0,TER,SNCF,16.821230,405
1,TRAIN,SNCF,12.039267,321
2,RER,SNCF,5.822027,208
3,METRO,RATP,2.854926,394
4,RER,RATP,1.242679,60


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui permet de créer des visualisations interactives et personnalisables. Elle est adaptée pour représenter des données complexes avec plusieurs variables, comme ici la longueur totale d'infrastructure par mode de transport et exploitant. Cela permet une exploration détaillée des données grâce aux informations affichées au survol.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px

df_sorted = df.sort_values("total_length", ascending=False)

dataviz = px.bar(
    df_sorted,
    x="mode",
    y="total_length",
    color="exploitant",
    hover_data=["num_lines", "total_length"],
    text_auto=False,
    title="Longueur totale d'infrastructure par mode de transport et exploitant",
    labels={"total_length": "Longueur (km)", "mode": "Mode de transport"},
    color_discrete_sequence=px.colors.qualitative.Pastel
)

dataviz.update_traces(
    hovertemplate="<b>%{x}</b><br>Longueur: %{customdata[0]:.2f} km<br>Lignes: %{customdata[1]}<extra></extra>"
)

dataviz.update_layout(
    hovermode="x unified",
    margin=dict(l=20, r=20, t=40, b=20),
    font=dict(family="Lato, sans-serif", size=11)
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250803_184642.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250803_184642.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250803_184642.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250803_184642.png


--> Image Plotly sauvegardée avec succès.
